In [1]:
import pandas as pd
import matplotlib
import theano
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
from neupy import algorithms, layers, init, environment, plots
from neupy.exceptions import StopTraining
from sklearn import datasets, cross_validation

ModuleNotFoundError: No module named 'theano'

In [ ]:
all_data=pd.read_csv("../all_data.csv", index_col=0)
all_data=np.array(all_data)
eps=0.000001
number_of_epochs=200

In [ ]:
bpnet=algorithms.GradientDescent(
     connection=[
         layers.Input(2),
         layers.Tanh(10, weight=init.XavierNormal(), bias=init.XavierNormal()),
         layers.Tanh(10, weight=init.XavierNormal(), bias=init.XavierNormal()),
         layers.Tanh(5, weight=init.XavierNormal(), bias=init.XavierNormal()),
         layers.Linear(1, weight=init.XavierNormal(), bias=init.XavierNormal()),
     ],
     error='mse',
     verbose=True,
     shuffle_data=True,
     show_epoch=50
     
)


In [ ]:
import copy
weight1=copy.deepcopy(bpnet.layers[1].weight.get_value())
bias1=copy.deepcopy(bpnet.layers[1].bias.get_value())
weight2=copy.deepcopy(bpnet.layers[2].weight.get_value())
bias2=copy.deepcopy(bpnet.layers[2].bias.get_value())
weight3=copy.deepcopy(bpnet.layers[3].weight.get_value())
bias3=copy.deepcopy(bpnet.layers[3].bias.get_value())
weight4=copy.deepcopy(bpnet.layers[4].weight.get_value())
bias4=copy.deepcopy(bpnet.layers[4].bias.get_value())

In [ ]:
test_errors=[]
def signal(net):
    test_errors.append(((net.predict(test_params)-test_target)**2).sum()/len(test_target))
    if len(net.errors)>20:
        if abs(net.errors[len(net.errors)-1]-net.errors[len(net.errors)-2]) < eps:
            raise StopTraining("Training has been interrupted")

In [ ]:
learn_error_task=[]
test_error_task=[]
for i in range(10):
    test_errors=[]
    np.random.seed(i)
    np.random.shuffle(all_data)
    learn_data = all_data[0: int(0.6*len(all_data))]
    validation_data = all_data[int(0.6*len(all_data)): int(0.9*len(all_data))]
    test_data = all_data[int(0.9*len(all_data)): len(all_data)]
    learn_params=learn_data[:,[0,1]]
    learn_target=learn_data[:,[2]]
    test_params=test_data[:,[0,1]]
    test_target=test_data[:,[2]]
    validation_params=validation_data[:,[0,1]]
    validation_target=validation_data[:,[2]]
    bpnet=algorithms.GradientDescent(
     connection=[
         layers.Input(2),
         layers.Tanh(10, weight=copy.deepcopy(weight1), bias=copy.deepcopy(bias1)),
         layers.Tanh(10, weight=copy.deepcopy(weight2), bias=copy.deepcopy(bias2)),
         layers.Tanh(5, weight=copy.deepcopy(weight3), bias=copy.deepcopy(bias3)),
         layers.Linear(1, weight=copy.deepcopy(weight4), bias=copy.deepcopy(bias4)),
     ],
     error='mse',
     step=0.05,
     epoch_end_signal=signal,
     verbose=True,
     shuffle_data=True,
     show_epoch=50 
    )
    bpnet.train(learn_params, learn_target, validation_params, validation_target, epochs=number_of_epochs)
    learn_error_task.append(bpnet.errors.last())
    test_error_task.append(test_errors[len(test_errors)-1])
print("Monte karlo learn error mean "+str(np.array(learn_error_task).mean()))
print("Monte karlo learn error std "+str(np.array(learn_error_task).std()))
print("Monte karlo test error mean "+str(np.array(test_error_task).mean()))
print("Monte karlo test error std "+str(np.array(test_error_task).std()))
    


In [ ]:
test_errors=[]
np.random.seed(42)
np.random.shuffle(all_data)
learn_data = all_data[0: int(0.6*len(all_data))]
validation_data = all_data[int(0.6*len(all_data)): int(0.9*len(all_data))]
test_data = all_data[int(0.9*len(all_data)): len(all_data)]
learn_params=learn_data[:,[0,1]]
learn_target=learn_data[:,[2]]
test_params=test_data[:,[0,1]]
test_target=test_data[:,[2]]
validation_params=validation_data[:,[0,1]]
validation_target=validation_data[:,[2]]
bpnet=algorithms.GradientDescent(
    connection=[
     layers.Input(2),
     layers.Tanh(10, weight=copy.deepcopy(weight1), bias=copy.deepcopy(bias1)),
     layers.Tanh(10, weight=copy.deepcopy(weight2), bias=copy.deepcopy(bias2)),
     layers.Tanh(5, weight=copy.deepcopy(weight3), bias=copy.deepcopy(bias3)),
     layers.Linear(1, weight=copy.deepcopy(weight4), bias=copy.deepcopy(bias4)),
    ],
    error='mse',
    step=0.05,
    epoch_end_signal=signal,
    verbose=True,
    shuffle_data=True,
    show_epoch=50 
)
bpnet.train(learn_params, learn_target, validation_params, validation_target, epochs=number_of_epochs)
print("Holdout learn error "+str(bpnet.errors.last()))
print("Holdout test error "+str(test_errors[len(test_errors)-1]))


In [ ]:
np.random.seed(57)
np.random.shuffle(all_data)
learn_error_task=[]
test_error_task=[]
for train_indices, test_indices in cross_validation.KFold(len(all_data), n_folds=10, shuffle=True):
    test_errors=[]
    tmp_data=all_data[train_indices]
    test_data=all_data[test_indices]
    learn_data=tmp_data[0:int(0.7*len(tmp_data))]
    validation_data=tmp_data[int(0.7*len(tmp_data)):len(tmp_data)]
    learn_params=learn_data[:,[0,1]]
    learn_target=learn_data[:,[2]]
    test_params=test_data[:,[0,1]]
    test_target=test_data[:,[2]]
    validation_params=validation_data[:,[0,1]]
    validation_target=validation_data[:,[2]]
    bpnet=algorithms.GradientDescent(
        connection=[
         layers.Input(2),
         layers.Tanh(10, weight=copy.deepcopy(weight1), bias=copy.deepcopy(bias1)),
         layers.Tanh(10, weight=copy.deepcopy(weight2), bias=copy.deepcopy(bias2)),
         layers.Tanh(5, weight=copy.deepcopy(weight3), bias=copy.deepcopy(bias3)),
         layers.Linear(1, weight=copy.deepcopy(weight4), bias=copy.deepcopy(bias4)),
        ],
        error='mse',
        step=0.05,
        epoch_end_signal=signal,
        verbose=True,
        shuffle_data=True,
        show_epoch=50 
    )
    bpnet.train(learn_params, learn_target, validation_params, validation_target, epochs=number_of_epochs)
    learn_error_task.append(bpnet.errors.last())
    test_error_task.append(test_errors[len(test_errors)-1])
print("10 fold learn error mean "+str(np.array(learn_error_task).mean()))
print("10 fold learn error std "+str(np.array(learn_error_task).std()))
print("10 fold test error mean "+str(np.array(test_error_task).mean()))
print("10 fold test error std "+str(np.array(test_error_task).std()))    
    

In [ ]:
learn_error_mean=[]
learn_error_std=[]
test_error_mean=[]
test_error_std=[]
fold_list=[2, 3, 4, 5, 7, 10, 13, 15, 20, 25]
for K in fold_list:
    np.random.shuffle(all_data)
    learn_error_task=[]
    test_error_task=[]
    for train_indices, test_indices in cross_validation.KFold(len(all_data), n_folds=K, shuffle=True):
        test_errors=[]
        tmp_data=all_data[train_indices]
        test_data=all_data[test_indices]
        learn_data=tmp_data[0:int(0.7*len(tmp_data))]
        validation_data=tmp_data[int(0.7*len(tmp_data)):len(tmp_data)]
        learn_params=learn_data[:,[0,1]]
        learn_target=learn_data[:,[2]]
        test_params=test_data[:,[0,1]]
        test_target=test_data[:,[2]]
        validation_params=validation_data[:,[0,1]]
        validation_target=validation_data[:,[2]]
        bpnet=algorithms.GradientDescent(
            connection=[
             layers.Input(2),
             layers.Tanh(10, weight=copy.deepcopy(weight1), bias=copy.deepcopy(bias1)),
             layers.Tanh(10, weight=copy.deepcopy(weight2), bias=copy.deepcopy(bias2)),
             layers.Tanh(5, weight=copy.deepcopy(weight3), bias=copy.deepcopy(bias3)),
             layers.Linear(1, weight=copy.deepcopy(weight4), bias=copy.deepcopy(bias4)),
            ],
            error='mse',
            step=0.05,
            epoch_end_signal=signal,
            verbose=True,
            shuffle_data=True,
            show_epoch=50 
        )
        bpnet.train(learn_params, learn_target, validation_params, validation_target, epochs=number_of_epochs)
        learn_error_task.append(bpnet.errors.last())
        test_error_task.append(test_errors[len(test_errors)-1])
    learn_error_mean.append(np.array(learn_error_task).mean())
    learn_error_std.append(np.array(learn_error_task).std())
    test_error_mean.append(np.array(test_error_task).mean())
    test_error_std.append(np.array(test_error_task).std())  
plt.clf()
plt.grid(True)
plt.plot(fold_list, learn_error_mean, color='r', label='train mean') 
plt.plot(fold_list,learn_error_std, color='b', label='train std') 
plt.plot(fold_list, test_error_mean, color='g', label='test mean')
plt.plot(fold_list, test_error_std, color='k', label='test std')
plt.legend()
plt.savefig('k_fold_test.png', bbox_inches='tight')

In [ ]:
np.random.seed(87)
np.random.shuffle(all_data)
learn_error_task=[]
test_error_task=[]
for train_indices, test_indices in cross_validation.LeaveOneOut(len(all_data)):
    test_errors=[]
    tmp_data=all_data[train_indices]
    test_data=all_data[test_indices]
    learn_data=tmp_data[0:int(0.7*len(tmp_data))]
    validation_data=tmp_data[int(0.7*len(tmp_data)):len(tmp_data)]
    learn_params=learn_data[:,[0,1]]
    learn_target=learn_data[:,[2]]
    test_params=test_data[:,[0,1]]
    test_target=test_data[:,[2]]
    validation_params=validation_data[:,[0,1]]
    validation_target=validation_data[:,[2]]
    bpnet=algorithms.GradientDescent(
        connection=[
         layers.Input(2),
         layers.Tanh(10, weight=copy.deepcopy(weight1), bias=copy.deepcopy(bias1)),
         layers.Tanh(10, weight=copy.deepcopy(weight2), bias=copy.deepcopy(bias2)),
         layers.Tanh(5, weight=copy.deepcopy(weight3), bias=copy.deepcopy(bias3)),
         layers.Linear(1, weight=copy.deepcopy(weight4), bias=copy.deepcopy(bias4)),
        ],
        error='mse',
        step=0.05,
        epoch_end_signal=signal,
        verbose=True,
        shuffle_data=True,
        show_epoch=50 
    )
    bpnet.train(learn_params, learn_target, validation_params, validation_target, epochs=number_of_epochs)
    learn_error_task.append(bpnet.errors.last())
    test_error_task.append(test_errors[len(test_errors)-1])
print("Loocv learn error mean "+str(np.array(learn_error_task).mean()))
print("Loocv learn error std "+str(np.array(learn_error_task).std()))
print("Loocv test error mean "+str(np.array(test_error_task).mean()))
print("Loocv test error std "+str(np.array(test_error_task).std()))    